<a href="https://colab.research.google.com/github/S-Asghari/Fact-Checking-Using-LLMs/blob/main/fine-tuning/split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets -q
from datasets import load_dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.9 MB/s eta 0:00:00


In [ ]:
labels = load_dataset("fever", name='v1.0', split='train')
labels_df = labels.to_pandas()
true_false_claims = labels_df[(labels_df.label=='REFUTES') | (labels_df.label=='SUPPORTS')][['claim', 'evidence_wiki_url', 'label']].copy()
del labels_df
true_false_claims['label'] = true_false_claims['label'].map({'SUPPORTS': True, 'REFUTES': False})

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/311431 [00:00<?, ? examples/s]

Generating labelled_dev split:   0%|          | 0/37566 [00:00<?, ? examples/s]

Generating unlabelled_dev split:   0%|          | 0/19998 [00:00<?, ? examples/s]

Generating unlabelled_test split:   0%|          | 0/19998 [00:00<?, ? examples/s]

Generating paper_dev split:   0%|          | 0/18999 [00:00<?, ? examples/s]

Generating paper_test split:   0%|          | 0/18567 [00:00<?, ? examples/s]

Dataset fever downloaded and prepared to /root/.cache/huggingface/datasets/fever/v1.0/1.0.0/7f8936e0558704771b08c7ce9cc202071b29a0050603374507ba61d23c00a58e. Subsequent calls will reuse this data.


In [ ]:
# bart_set, bertAndTestSet = train_test_split(true_false_claims, test_size=0.4, random_state=42)
# bert_set, test_set = train_test_split(bertAndTestSet, test_size=0.25, random_state=42)

In [ ]:
groups = true_false_claims.groupby('evidence_wiki_url')
wiki_urls = true_false_claims['evidence_wiki_url'].unique()
np.random.shuffle(wiki_urls)

total_set_size = len(true_false_claims)
bart_set_size = int(total_set_size * 0.6)
bert_set_size = int(total_set_size * 0.3)
test_set_size = total_set_size - (bart_set_size + bert_set_size)

bart_set = pd.DataFrame()
bert_set = pd.DataFrame()
test_set = pd.DataFrame()

for url in wiki_urls:
    group = groups.get_group(url)

    if len(bart_set) < bart_set_size:
        bart_set = pd.concat([bart_set, group])
    elif len(bert_set) < bert_set_size:
        bert_set = pd.concat([bert_set, group])
    else:
        test_set = pd.concat([test_set, group])

bart_set.reset_index(drop=True, inplace=True)
bert_set.reset_index(drop=True, inplace=True)
test_set.reset_index(drop=True, inplace=True)

In [ ]:
# print("Whole dataset size:",len(true_false_claims))
# print("BART set size:", len(bart_set))
# print("BERT set size:", len(bert_set))
# print("Test set size:", len(test_set))

In [ ]:
del true_false_claims

In [ ]:
bart_set2 = bart_set[bart_set.label== True].copy()
del bart_set
# print("New BART set size:", len(bart_set2))

In [ ]:
wiki = load_dataset("fever", name='wiki_pages')
wiki_df = wiki['wikipedia_pages'].to_pandas()
wiki_id_text_df = wiki_df[['id', 'text']].copy()
del wiki_df
wiki_id_text_df.columns = ['evidence_wiki_url', 'text']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating wikipedia_pages split:   0%|          | 0/5416537 [00:00<?, ? examples/s]

Dataset fever downloaded and prepared to /root/.cache/huggingface/datasets/fever/wiki_pages/1.0.0/7f8936e0558704771b08c7ce9cc202071b29a0050603374507ba61d23c00a58e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
join_bart_df = pd.merge(bart_set2, wiki_id_text_df, on='evidence_wiki_url')
del bart_set2
join_bart_df = join_bart_df.drop('evidence_wiki_url', axis=1)
join_bart_df = join_bart_df.drop_duplicates().reset_index(drop=True)


join_bert_df = pd.merge(bert_set, wiki_id_text_df, on='evidence_wiki_url')
del bert_set
join_bert_df = join_bert_df.drop('evidence_wiki_url', axis=1)
join_bert_df = join_bert_df.drop_duplicates().reset_index(drop=True)

join_test_df = pd.merge(test_set, wiki_id_text_df, on='evidence_wiki_url')
del test_set
del wiki_id_text_df
join_test_df = join_test_df.drop('evidence_wiki_url', axis=1)
join_test_df = join_test_df.drop_duplicates().reset_index(drop=True)

In [ ]:
join_bart_df.to_csv('bart_set.csv')
join_bert_df.to_csv('bert_set.csv')
join_test_df.to_csv('test_set.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r 'bart_set.csv' /content/drive/MyDrive/DeClare/
!cp -r 'bert_set.csv' /content/drive/MyDrive/DeClare/
!cp -r 'test_set.csv' /content/drive/MyDrive/DeClare/